In [2]:

import torch
# from train_resnet import SelfNet
import os
import os.path as osp
import shutil
import torch.nn as nn
from PIL import Image
import timm
from torchvision import transforms

# set to gpu
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model_path = "./savefile/vit_tiny_patch16_224_pretrained_224/vit_tiny_patch16_224_9epochs_accuracy0.99922_weights.pth"
# 'A', 'B', 'Blank', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "vit_tiny_patch16_224"
num_classes = len(classes_names)

# ---------

# define the model
class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes,
                 pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)  # load pretrained model
        # classifier
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features  # fix the connect layer number
            self.model.fc = nn.Linear(n_features, out_features)  # change the class numebr
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features  # change full connect layer number
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)
        # resnet  change the all connect layer
    #     print(self.model)  # return model

    def forward(self, x):  # front spreed
        x = self.model(x)
        return x


def get_torch_transforms(img_size=224):
    data_transforms = {
        'train': transforms.Compose([
            # transforms.RandomResizedCrop(img_size),
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(p=0.2),
            transforms.RandomRotation((-5, 5)),
            transforms.RandomAutocontrast(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            # transforms.Resize((img_size, img_size)),
            # transforms.Resize(256),
            # transforms.CenterCrop(img_size),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    # Convert input image to RGB format
    # data_transforms['val'].transforms.insert(0, transforms.Grayscale(3))
    return data_transforms

# ---------


def predict_batch(model_path, target_dir, save_dir):
    data_transforms = get_torch_transforms(img_size=img_size)
    valid_transforms = data_transforms['val']
    # load the net
    model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
    # model = nn.DataParallel(model)
    weights = torch.load(model_path)
    model.load_state_dict(weights)
    model.eval()
    model.to(device)
    # read picture
    image_names = os.listdir(target_dir)
    for i, image_name in enumerate(image_names):
        image_path = osp.join(target_dir, image_name)
        img = Image.open(image_path)
        img = valid_transforms(img)
        img = img.unsqueeze(0)
        img = img.to(device)
        output = model(img)
        label_id = torch.argmax(output).item()
        predict_name = classes_names[label_id]
        save_path = osp.join(save_dir, predict_name)
        if not osp.isdir(save_path):
            os.makedirs(save_path)
        shutil.copy(image_path, save_path)
        print(f"{i + 1}: {image_name} result {predict_name}")


def predict_single(model_path, image_path):
    data_transforms = get_torch_transforms(img_size=img_size)
    # train_transforms = data_transforms['train']
    valid_transforms = data_transforms['val']
    # load the net
    model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
    # model = nn.DataParallel(model)
    weights = torch.load(model_path)
    model.load_state_dict(weights)
    model.eval()
    model.to(device)

    # read picture
    img = Image.open(image_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = valid_transforms(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    output = model(img)
    label_id = torch.argmax(output).item()
    predict_name = classes_names[label_id]
    print(f"{image_path}'s result is {predict_name}")


if __name__ == '__main__':
    # bacth predict

    # target_dir="./gesture_finalpredict/asl_alphabet_test"
    # predict_batch(model_path=model_path,target_dir="./americanSignGesture/professorhand/gesture_finalpredict/test2",save_dir="./americanSignGesture/professorhand/gesture_finalpredict/theresultfortest2")

    predict_batch(model_path=model_path,target_dir="./gesture_finalpredict/asl_alphabet_test",save_dir="./gesture_finalpredict/theresultfortest2")
    predict_batch(model_path=model_path,target_dir="./gesture_finalpredict/test2",save_dir="./gesture_finalpredict/theresultfortest2")

    # single predict
    #predict_single(model_path=model_path, image_path="123.jpg")

    # predict_single(model_path=model_path, image_path="2.jpg")

# predict_single(model_path=model_path, image_path="3.jpg")

#  predict_single(model_path=model_path, image_path="4.jpg")

#  predict_single(model_path=model_path, image_path="5.jpg")

#   predict_single(model_path=model_path, image_path="6.jpg")

#  predict_single(model_path=model_path, image_path="7.jpg")

#  predict_single(model_path=model_path, image_path="./gesture_finalpredict/test2/b.jpg")


1: A_test.jpg result A
2: B_test.jpg result B
3: C_test.jpg result C
4: D_test.jpg result D
5: E_test.jpg result E
6: F_test.jpg result F
7: G_test.jpg result G
8: H_test.jpg result H
9: I_test.jpg result I
10: J_test.jpg result J
11: K_test.jpg result K
12: L_test.jpg result L
13: M_test.jpg result M
14: nothing_test.jpg result nothing
15: N_test.jpg result N
16: O_test.jpg result O
17: P_test.jpg result P
18: Q_test.jpg result Q
19: R_test.jpg result R
20: space_test.jpg result space
21: S_test.jpg result S
22: T_test.jpg result T
23: U_test.jpg result U
24: V_test.jpg result V
25: W_test.jpg result W
26: X_test.jpg result X
27: Y_test.jpg result Y
28: Z_test.jpg result Z
1: aa.jpg result Y
2: b.jpg result O
3: bb.jpg result space
4: bbbbbb.jpg result space
5: cc.jpg result C
6: cccc.jpg result F
7: dd.jpg result Y
8: dddd.jpg result F
9: ee.jpg result J
10: ff.jpg result O
11: gg.jpg result P
12: hh.jpg result H
13: jj.jpg result space
14: kk.jpg result Z
15: ll.jpg result space
16: